# EBAC - Regressão II - regressão múltipla

## Tarefa II

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [122]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import patsy
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [124]:
df = pd.read_csv('previsao_de_renda.csv')

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [129]:
df = df.drop(columns=["Unnamed: 0", "data_ref", "id_cliente"])

In [139]:
df["tempo_emprego"] = df["tempo_emprego"].fillna(df["tempo_emprego"].mean())

df = pd.get_dummies(df, drop_first=True)


df["renda"] = np.log(df["renda"])

x = df.drop(columns=["renda"])
y = df["renda"]

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   posse_de_veiculo               15000 non-null  bool   
 1   posse_de_imovel                15000 non-null  bool   
 2   qtd_filhos                     15000 non-null  int64  
 3   idade                          15000 non-null  int64  
 4   tempo_emprego                  15000 non-null  float64
 5   qt_pessoas_residencia          15000 non-null  float64
 6   renda                          15000 non-null  float64
 7   sexo_M                         15000 non-null  bool   
 8   tipo_renda_Bolsista            15000 non-null  bool   
 9   tipo_renda_Empresário          15000 non-null  bool   
 10  tipo_renda_Pensionista         15000 non-null  bool   
 11  tipo_renda_Servidor público    15000 non-null  bool   
 12  educacao_Pós graduação         15000 non-null 

In [143]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=123)

In [157]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_resultados = {}
for alpha in alphas:
    ridge = Ridge(alpha = alpha)
    ridge.fit(x_train, y_train)
    
    y_pred = ridge.predict(x_test)
    
    r2 = r2_score(y_test, y_pred)
    ridge_resultados[alpha] = r2

melhor_r2_ridge =  ridge_resultados[max(ridge_resultados, key = ridge_resultados.get)]
max(ridge_resultados, key = ridge_resultados.get), melhor_r2_ridge

(0, 0.3130166751462493)

O melhor modelo foi o modelo com o alpha em 0

In [159]:
lasso_resultados = {}
for alpha in alphas:
    lasso = Lasso(alpha = alpha, max_iter = 1000)
    lasso.fit(x_train, y_train)
    
    y_pred = lasso.predict(x_test)
    
    r2 = r2_score(y_test, y_pred)
    lasso_resultados[alpha] = r2

melhor_r2_lasso = lasso_resultados[max(lasso_resultados, key = lasso_resultados.get)]
max(lasso_resultados, key = lasso_resultados.get), melhor_r2_lasso

C:\Users\Usuario01\anaconda3\Lib\site-packages\sklearn\base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\Usuario01\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Usuario01\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.292e+01, tolerance: 1.297e-02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.en

(0, 0.31300851479103275)

O melhor modelo Lasso tambem foi com o alpha = 0

In [163]:
x_train = x_train.astype(int)
x_test = x_test.astype(int)

modelo01 =sm.OLS(y_train,x_train).fit()
y_pred = modelo01.predict(x_test)


In [165]:
modelo01.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  renda   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          2.038e+05
Date:                Thu, 30 Jan 2025   Prob (F-statistic):                        0.00
Time:                        19:26:54   Log-Likelihood:                          9869.3
No. Observations:               11250   AIC:                                 -1.969e+04
Df Residuals:                   11226   BIC:                                 -1.951e+04
Df Model:                          24                                                  
Covariance Type:            nonrobust                                                  
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
posse_de_veiculo                  0.0097      0.002      4.537      0.000       0.005       0.014
posse_de_imovel                   0.0144      0.002      6.847      0.000       0.010       0.019
qtd_filhos                       -0.8627      0.006   -133.519      0.000      -0.875      -0.850
idade                             0.0014      0.000     11.013      0.000       0.001       0.002
tempo_emprego                     0.0072      0.000     43.505      0.000       0.007       0.008
qt_pessoas_residencia             0.8707      0.006    141.378      0.000       0.859       0.883
sexo_M                            0.1002      0.002     44.770      0.000       0.096       0.105
tipo_renda_Bolsista               0.0309      0.034      0.918      0.359      -0.035       0.097
tipo_renda_Empresário             0.0239      0.002      9.968      0.000       0.019       0.029
tipo_renda_Pensionista           -0.0268      0.003     -7.809      0.000      -0.034      -0.020
tipo_renda_Servidor público       0.0097      0.004      2.686      0.007       0.003       0.017
educacao_Pós graduação            0.1149      0.026      4.429      0.000       0.064       0.166
educacao_Secundário               0.0905      0.009     10.530      0.000       0.074       0.107
educacao_Superior completo        0.1066      0.009     12.308      0.000       0.090       0.124
educacao_Superior incompleto      0.0917      0.010      9.348      0.000       0.072       0.111
estado_civil_Separado             0.8744      0.007    118.795      0.000       0.860       0.889
estado_civil_Solteiro             0.8691      0.007    132.031      0.000       0.856       0.882
estado_civil_União                0.0031      0.004      0.839      0.402      -0.004       0.011
estado_civil_Viúvo                0.8736      0.008    112.030      0.000       0.858       0.889
tipo_residencia_Casa              0.0994      0.008     11.891      0.000       0.083       0.116
tipo_residencia_Com os pais       0.1101      0.009     11.743      0.000       0.092       0.129
tipo_residencia_Comunitário       0.0945      0.016      5.808      0.000       0.063       0.126
tipo_residencia_Estúdio           0.1039      0.015      6.794      0.000       0.074       0.134
tipo_residencia_Governamental     0.1040      0.010     10.414      0.000       0.084       0.124
==============================================================================
Omnibus:                     9259.212   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3329027.077
Skew:                           2.961   Prob(JB):                         0.00
Kurtosis:             

In [167]:
r2 = r2_score(y_test, y_pred)
r2

0.09217613751636078

In [169]:
x_train.drop('estado_civil_União',axis = 1, inplace = True)
modelo02 = sm.OLS(y_train,x_train).fit()
modelo02.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  renda   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          2.126e+05
Date:                Thu, 30 Jan 2025   Prob (F-statistic):                        0.00
Time:                        19:26:55   Log-Likelihood:                          9868.9
No. Observations:               11250   AIC:                                 -1.969e+04
Df Residuals:                   11227   BIC:                                 -1.952e+04
Df Model:                          23                                                  
Covariance Type:            nonrobust                                                  
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
posse_de_veiculo                  0.0096      0.002      4.494      0.000       0.005       0.014
posse_de_imovel                   0.0144      0.002      6.835      0.000       0.010       0.018
qtd_filhos                       -0.8631      0.006   -133.827      0.000      -0.876      -0.850
idade                             0.0014      0.000     10.984      0.000       0.001       0.002
tempo_emprego                     0.0072      0.000     43.506      0.000       0.007       0.008
qt_pessoas_residencia             0.8709      0.006    141.656      0.000       0.859       0.883
sexo_M                            0.1002      0.002     44.764      0.000       0.096       0.105
tipo_renda_Bolsista               0.0306      0.034      0.909      0.363      -0.035       0.097
tipo_renda_Empresário             0.0240      0.002     10.007      0.000       0.019       0.029
tipo_renda_Pensionista           -0.0267      0.003     -7.797      0.000      -0.033      -0.020
tipo_renda_Servidor público       0.0097      0.004      2.678      0.007       0.003       0.017
educacao_Pós graduação            0.1146      0.026      4.420      0.000       0.064       0.165
educacao_Secundário               0.0904      0.009     10.527      0.000       0.074       0.107
educacao_Superior completo        0.1065      0.009     12.302      0.000       0.090       0.124
educacao_Superior incompleto      0.0917      0.010      9.346      0.000       0.072       0.111
estado_civil_Separado             0.8744      0.007    118.795      0.000       0.860       0.889
estado_civil_Solteiro             0.8691      0.007    132.030      0.000       0.856       0.882
estado_civil_Viúvo                0.8736      0.008    112.031      0.000       0.858       0.889
tipo_residencia_Casa              0.0994      0.008     11.897      0.000       0.083       0.116
tipo_residencia_Com os pais       0.1102      0.009     11.756      0.000       0.092       0.129
tipo_residencia_Comunitário       0.0944      0.016      5.799      0.000       0.062       0.126
tipo_residencia_Estúdio           0.1040      0.015      6.808      0.000       0.074       0.134
tipo_residencia_Governamental     0.1040      0.010     10.409      0.000       0.084       0.124
==============================================================================
Omnibus:                     9267.907   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3339169.173
Skew:                           2.965   Prob(JB):                         0.00
Kurtosis:                      87.193   Cond. No.                     1.63e+03
=========================================

In [171]:
x_train.drop('tipo_renda_Bolsista',axis = 1, inplace = True)
modelo03 = sm.OLS(y_train,x_train).fit()
modelo03.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  renda   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          2.223e+05
Date:                Thu, 30 Jan 2025   Prob (F-statistic):                        0.00
Time:                        19:26:56   Log-Likelihood:                          9868.5
No. Observations:               11250   AIC:                                 -1.969e+04
Df Residuals:                   11228   BIC:                                 -1.953e+04
Df Model:                          22                                                  
Covariance Type:            nonrobust                                                  
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
posse_de_veiculo                  0.0095      0.002      4.476      0.000       0.005       0.014
posse_de_imovel                   0.0144      0.002      6.851      0.000       0.010       0.019
qtd_filhos                       -0.8631      0.006   -133.826      0.000      -0.876      -0.850
idade                             0.0014      0.000     11.010      0.000       0.001       0.002
tempo_emprego                     0.0072      0.000     43.507      0.000       0.007       0.008
qt_pessoas_residencia             0.8709      0.006    141.654      0.000       0.859       0.883
sexo_M                            0.1001      0.002     44.757      0.000       0.096       0.105
tipo_renda_Empresário             0.0239      0.002      9.987      0.000       0.019       0.029
tipo_renda_Pensionista           -0.0269      0.003     -7.835      0.000      -0.034      -0.020
tipo_renda_Servidor público       0.0096      0.004      2.661      0.008       0.003       0.017
educacao_Pós graduação            0.1146      0.026      4.421      0.000       0.064       0.165
educacao_Secundário               0.0904      0.009     10.527      0.000       0.074       0.107
educacao_Superior completo        0.1066      0.009     12.312      0.000       0.090       0.124
educacao_Superior incompleto      0.0917      0.010      9.349      0.000       0.072       0.111
estado_civil_Separado             0.8743      0.007    118.794      0.000       0.860       0.889
estado_civil_Solteiro             0.8690      0.007    132.030      0.000       0.856       0.882
estado_civil_Viúvo                0.8735      0.008    112.029      0.000       0.858       0.889
tipo_residencia_Casa              0.0994      0.008     11.897      0.000       0.083       0.116
tipo_residencia_Com os pais       0.1102      0.009     11.755      0.000       0.092       0.129
tipo_residencia_Comunitário       0.0943      0.016      5.796      0.000       0.062       0.126
tipo_residencia_Estúdio           0.1040      0.015      6.807      0.000       0.074       0.134
tipo_residencia_Governamental     0.1040      0.010     10.407      0.000       0.084       0.124
==============================================================================
Omnibus:                     9266.148   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3337562.580
Skew:                           2.964   Prob(JB):                         0.00
Kurtosis:                      87.172   Cond. No.                     1.31e+03
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) si

In [182]:
x_test_ajustado = x_test.drop(['estado_civil_União','tipo_renda_Bolsista'],axis = 1)
y_pred_stepwise =  modelo03.predict(x_test_ajustado)

r2_stepwise = r2_score(y_test, y_pred_stepwise)
r2_stepwise

0.09239011100430239

Dos 3 modelos treinados, o modelo Lasso e o modelo Ridge obtiveram os melhores resultados na base de testes ambos com alpha em 0. com R2 aproximado em 0.3130

Por fims de simplificação será utilizado o modelo Lasso apos a transformação das variaveis a fim de obter R2 maiores.

In [195]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state = 100)

x_train['idade_tempo_emprego'] = x_train['idade'] * x_train['tempo_emprego']
x_test['idade_tempo_emprego'] = x_test['idade'] * x_test['tempo_emprego']


y_train_log = np.log(y_train) 

lasso = Lasso(alpha = 0, max_iter = 1000)

lasso.fit(x_train, y_train_log)


y_pred = np.exp(lasso.predict(x_test))

r2_improved = r2_score(y_test, y_pred)s
r2_improved

C:\Users\Usuario01\anaconda3\Lib\site-packages\sklearn\base.py:1474: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\Usuario01\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Usuario01\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.003e+01, tolerance: 2.977e-03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.en

0.33240516613509885

In [211]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

tree = DecisionTreeRegressor(random_state=42, max_depth = 5) 
tree.fit(x_train, y_train)

y_pred_tree = tree.predict(x_test)
r2_tree = r2_score(y_test, y_pred_tree)

r2_tree

0.3256086161138686

Foi possivel chegar em um r2 maior com uma arvore de regressão, porem a diferença foi pequena, cerca de 0.01.